**Table of contents**<a id='toc0_'></a>    
- [Data Mining Lab 2 - Phase 2](#toc1_)    
  - [Before Starting](#toc1_1_)    
  - [Introduction](#toc1_2_)    
  - [**1. Data Preparation**](#toc1_3_)    
  - [**1.1 Load data**](#toc1_4_)    
    - [**1.2 Save data**](#toc1_4_1_)    
  - [**2. Large Language Models (LLMs)**](#toc1_5_)    
    - [Open-Source vs. Proprietary LLMs](#toc1_5_1_)    
    - [Why Use Code (API) for Data Mining?](#toc1_5_2_)    
    - [The Gemini API](#toc1_5_3_)    
    - [Interacting with the Gemini API](#toc1_5_4_)    
    - [**2.1 Text Prompting**](#toc1_5_5_)    
        - [**>>> Exercise 1 (Take home):**](#toc1_5_5_1_1_)    
    - [**2.2 Structured Output**](#toc1_5_6_)    
        - [**>>> Exercise 2 (Take home):**](#toc1_5_6_1_1_)    
    - [**2.3 Information Extraction and Grounding:**](#toc1_5_7_)    
      - [**`langextract`: A Library for Grounded Extraction**](#toc1_5_7_1_)    
        - [**2.3.1 Using PDF Documents:**](#toc1_5_7_1_1_)    
        - [**>>> Bonus Exercise 3 (Take home):**](#toc1_5_7_1_2_)    
    - [**2.4 Generating LLM Embeddings:**](#toc1_5_8_)    
        - [**>>> Exercise 4 (Take home):**](#toc1_5_8_1_1_)    
    - [**2.5 Retrieval-Augmented Generation (RAG)**](#toc1_5_9_)    
        - [**Actual answer in the URL:**](#toc1_5_9_1_1_)    
        - [**Content in the URL that might get into the generated answer because of similar semantic meaning:**](#toc1_5_9_1_2_)    
        - [**>>> Bonus Exercise 5 (Take home):**](#toc1_5_9_1_3_)    
    - [**2.6 Few-Shot Prompting Classification:**](#toc1_5_10_)    
        - [**>>> Exercise 6 (Take home):**](#toc1_5_10_1_1_)    
        - [**>>> Exercise 7 (Take home):**](#toc1_5_10_1_2_)    
    - [**2.7 Extra LLM Related Materials:**](#toc1_5_11_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Data Mining Lab 2 - Phase 2](#toc0_)
In this lab's phase 2 session we will focus on exploring some basic LLMs' applications with data.


## <a id='toc1_1_'></a>[Before Starting](#toc0_)

**Make sure you have installed all the required libraries and you have the environment ready to run this lab.**
    

---
## <a id='toc1_2_'></a>[Introduction](#toc0_)

**Dataset:** [SemEval 2017 Task](https://competitions.codalab.org/competitions/16380)

**Task:** Classify text data into 4 different emotions using word embeddings and other deep information retrieval approaches.

![pic0.png](./pics/pic0.png)

---
## <a id='toc1_3_'></a>[**1. Data Preparation**](#toc0_)

---
## <a id='toc1_4_'></a>[**1.1 Load data**](#toc0_)

We start by loading the csv files into a single pandas dataframe for training and one for testing.

In [89]:
import pandas as pd

### training data
anger_train = pd.read_csv("data/semeval/train/anger-ratings-0to1.train.txt",
                         sep="\t", header=None,names=["id", "text", "emotion", "intensity"])
sadness_train = pd.read_csv("data/semeval/train/sadness-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
fear_train = pd.read_csv("data/semeval/train/fear-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
joy_train = pd.read_csv("data/semeval/train/joy-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])

In [90]:
# combine 4 sub-dataset
train_df = pd.concat([anger_train, fear_train, joy_train, sadness_train], ignore_index=True)

In [91]:
### testing data
anger_test = pd.read_csv("data/semeval/dev/anger-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
sadness_test = pd.read_csv("data/semeval/dev/sadness-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
fear_test = pd.read_csv("data/semeval/dev/fear-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
joy_test = pd.read_csv("data/semeval/dev/joy-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])

# combine 4 sub-dataset
test_df = pd.concat([anger_test, fear_test, joy_test, sadness_test], ignore_index=True)
train_df.head()

,id,text,emotion,intensity
0,10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938
1,10001,So my Indian Uber driver just called someone t...,anger,0.896
2,10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896
4,10004,Don't join @BTCare they put the phone down on ...,anger,0.896


In [92]:
# shuffle dataset
train_df = train_df.sample(frac=1)
test_df = test_df.sample(frac=1)

In [93]:
print("Shape of Training df: ", train_df.shape)
print("Shape of Testing df: ", test_df.shape)

Shape of Training df:  (3613, 4)
Shape of Testing df:  (347, 4)


---
### <a id='toc1_4_1_'></a>[**1.2 Save data**](#toc0_)

In [94]:
# save to pickle file
train_df.to_pickle("./data/train_df.pkl") 
test_df.to_pickle("./data/test_df.pkl")

In [95]:
import pandas as pd

# load a pickle file
train_df = pd.read_pickle("./data/train_df.pkl")
test_df = pd.read_pickle("./data/test_df.pkl")

For more information: https://reurl.cc/0Dzqx

---
## <a id='toc1_5_'></a>[**2. Large Language Models (LLMs)**](#toc0_)

Before we start we strongly suggest that you watch the following video explanations so you can understand the concepts that we are gonna discuss about LLMs: 

1. [How Large Language Models Work](https://www.youtube.com/watch?v=5sLYAQS9sWQ)
2. [Large Language Models explained briefly](https://www.youtube.com/watch?v=LPZh9BOjkQs)
3. [What is Prompt Tuning?](https://www.youtube.com/watch?v=yu27PWzJI_Y)
4. [Why Large Language Models Hallucinate](https://www.youtube.com/watch?v=cfqtFvWOfg0)
5. [What are LLM Embeddings?](https://www.youtube.com/watch?v=UShw_1NbpCw&t=182s)
6. [What is Retrieval-Augmented Generation (RAG)?](https://www.youtube.com/watch?v=T-D1OfcDW1M)
7. [RAG vs Fine-Tuning vs Prompt Engineering: Optimizing AI Models](https://www.youtube.com/watch?v=zYGDpG-pTho)
8. [Discover Few-Shot Prompting | Google AI Essentials](https://www.youtube.com/watch?v=9qdgEBVkWR4)
9. [What is Zero-Shot Learning?](https://www.youtube.com/watch?v=pVpr4GYLzAo)
10. [Zero-shot, One-shot and Few-shot Prompting Explained | Prompt Engineering 101](https://www.youtube.com/watch?v=sW5xoicq5TY)

`These videos can help you get a better grasp on the core concepts of LLMs if you were not familiar before.`

**So now let's start with the main content of Lab 2 Phase 2.**

Large Language Models (LLMs) are AI systems trained on vast amounts of text to understand and generate human language for tasks like summarization and translation.

### <a id='toc1_5_1_'></a>[Open-Source vs. Proprietary LLMs](#toc0_)
*   **Open-Source Models** (e.g., Llama, Gemma) are customizable and cost-effective but require technical skill to manage and may be less powerful.
*   **Proprietary Models** (e.g., Gemini, ChatGPT) offer top performance and ease of use but are more costly and less flexible.

For students interested in running models locally, the optional notebook `DM2025-Lab2-Optional-Ollama.ipynb` explores using Ollama ([Ollama GitHub Link](https://github.com/ollama/ollama)). It needs a capable GPU to run models (**at least 4GB VRAM**).

You can explore the variety of models available through Ollama here:

![pic10.png](./pics/pic10.png)

### <a id='toc1_5_2_'></a>[Why Use Code (API) for Data Mining?](#toc0_)

For data analysis, accessing LLMs programmatically is superior to using web chatbots because it allows for:
*   **Automation:** Easily process entire datasets with loops.
*   **Structured Output:** Receive data in usable formats like **JSON**, ready for analysis in tools like pandas.
*   **Reproducibility:** Ensure consistent results by setting fixed parameters.
*   **Privacy:** Maintain data security, especially when running models locally.

For the main exercises in this lab, we will use **the Gemini API**. This approach offers several advantages over running local open-source models, such as access to state-of-the-art model performance without needing specialized hardware. While the API has usage limits (rate limits and token quotas), it provides a generous **free tier** that is more than sufficient for our exercises.

![pic13.png](./pics/pic13.png)

![pic14.png](./pics/pic14.png)

### <a id='toc1_5_3_'></a>[The Gemini API](#toc0_)

We will primarily use the **Gemini 2.5 Flash-Lite** (`gemini-2.5-flash-lite`) model. As shown in the rate limit table, this model is optimized for high-frequency tasks and offers a high request-per-day limit of 1,000, making it ideal for completing the lab exercises without interruption.

Students are encouraged to explore other models available through the API but should remain mindful of their respective usage limits. For instance:
*   **Gemini 2.5 Pro** is a more powerful model but has a lower daily request limit of 100.
*   The **Gemma 3** model available via the API offers an impressive 14,400 requests per day, providing another excellent alternative for experimentation.

Please be aware of your usage limits as you work through the exercises to ensure you do not get rate-limited.

[Gemini Documentation](https://ai.google.dev/gemini-api/docs)

[Gemini Rate Limits](https://ai.google.dev/gemini-api/docs/rate-limits)

[Description of Gemini Models](https://ai.google.dev/gemini-api/docs/models)

---

### <a id='toc1_5_4_'></a>[Interacting with the Gemini API](#toc0_)

The code cell below contains the primary function, `prompt_gemini`, that we will use throughout this lab to communicate with the Gemini API. It's designed to be a flexible wrapper that handles the details of sending a request and receiving a response.

Before you run the exercises, here are the key things you need to understand in this setup:

*   **API Key Configuration**: The script loads your API key from a `.env` file located in the `./config/` directory. **You must create this file and add your API key** like this: `GOOGLE_API_KEY='YOUR_API_KEY_HERE'`. This is a security best practice to keep your credentials out of the code.

*   **Global Settings**: At the top of the script, you can find and modify several important defaults:
    *   `MODEL_NAME`: We've set this to `"gemini-2.5-flash-lite"`, but you can easily switch to other models like `"gemini-2.5-pro"` to experiment.
    *   `SYSTEM_INSTRUCTION`: This sets the model's default behavior or persona (e.g., "You are a helpful assistant"). You can customize this for different tasks.
    *   `SAFETY_SETTINGS`: For our academic exercises, these are turned off to prevent interference. In real-world applications, you would configure these carefully.

*   **The `prompt_gemini` function**: This is the main tool you will use. Here are its most important parameters:
    *   `input_prompt`: The list of contents (text, images, etc.) you want to send to the model.
    *   `temperature`: Controls the randomness of the output. `0.0` makes the output deterministic and less creative, while a higher value (e.g., `0.7`) makes it more varied.
    *   `schema`: A powerful feature that allows you to specify a JSON format for the model's output. This is extremely useful for structured data extraction.
    *   `with_tokens_info`: If set to `True`, the function will also return the number of input and output tokens used, which is helpful for monitoring your usage against the free tier limits.

In the following exercises, you will call this function with different prompts and configurations to solve various tasks.

If needed, you can also check some tutorials on how a python function works: [Python Functions Tutorial](https://realpython.com/defining-your-own-python-function/)

In [96]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

env_path = "./config/.env"
load_dotenv(dotenv_path=env_path)

# System instruction that can dictate how the model behaves in the output, can be customized as needed
SYSTEM_INSTRUCTION = (
        "You are a helpful assistant"
    )

# Max amount of tokens that the model can output, the Gemini 2.5 Models have this maximum amount
# For other models need to check their documentation 
MAX_OUTPUT_TOKENS = 65535
MODEL_NAME = "gemini-2.5-flash-lite" # Other models: "gemini-2.5-pro", "gemini-2.5-flash"; Check different max output tokens: "gemini-2.0-flash" , "gemini-2.0-flash-lite" 

# We disable the safety settings, as no moderation is needed in our tasks
SAFETY_SETTINGS = [
    types.SafetySetting(
        category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
    types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
    types.SafetySetting(
        category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
    types.SafetySetting(
        category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
]

#IMPORTANT: The script loads your API key from a `.env` file located in the `./config/` directory. 
# You must create this file and add your API key like this: `GOOGLE_API_KEY='YOUR_API_KEY_HERE'`

# We input the API Key to be able to use the Gemini models
api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = api_key
client = genai.Client(api_key=api_key)

# We also set LangExtract to use the API key as well:
if 'GEMINI_API_KEY' not in os.environ:
    os.environ['GEMINI_API_KEY'] = api_key

def prompt_gemini(
        input_prompt: list,
        schema = None,
        temperature: float = 0.0,
        system_instruction: str = SYSTEM_INSTRUCTION,
        max_output_tokens: int = MAX_OUTPUT_TOKENS,
        client: genai.Client = client,
        model_name: str = MODEL_NAME,
        new_config: types.GenerateContentConfig = None,
        with_tools: bool = False,
        with_parts: bool = False,
        with_tokens_info: bool = False
    ):
        try:
            # If we need a JSON schema we set up the following
            if schema:
                generate_content_config = types.GenerateContentConfig(
                    temperature=temperature,
                    system_instruction=system_instruction,
                    max_output_tokens=max_output_tokens,
                    response_modalities=["TEXT"],
                    response_mime_type="application/json",
                    response_schema=schema,
                    safety_settings=SAFETY_SETTINGS
                )
            # If there is no need we leave it unstructured
            else:
                generate_content_config = types.GenerateContentConfig(
                    temperature=temperature,
                    system_instruction=system_instruction,
                    max_output_tokens=max_output_tokens,
                    response_modalities=["TEXT"],
                    safety_settings=SAFETY_SETTINGS
                )
            
            # We add a different custom configuration if we need it
            if new_config:
                generate_content_config = new_config
            
            # For some tasks we need a more specific way to add the contents when prompting the model
            # So we need custom parts for it sometimes from the "types" objects
            if with_parts:
                response = client.models.generate_content(
                    model=model_name,
                    contents=types.Content(parts=input_prompt),
                    config=generate_content_config,
                )
            # In the simplest form the contents can be expressed as a list [] of simple objects like str and Pillow images
            else:
                response = client.models.generate_content(
                    model=model_name,
                    contents=input_prompt,
                    config=generate_content_config,
                )

            if with_tools:
                # print(response)
                # Include raw response when function calling
                completion = response
                if with_tokens_info:
                    log = {
                        "model": model_name,
                        "input_tokens": response.usage_metadata.prompt_token_count,
                        "output_tokens": response.usage_metadata.candidates_token_count,
                    }
                    return completion, log
                return completion
            else:
                completion = response.text
                if with_tokens_info:
                    log = {
                        "model": model_name,
                        "input_tokens": response.usage_metadata.prompt_token_count,
                        "output_tokens": response.usage_metadata.candidates_token_count,
                    }
                    # Return the text response and logs (if selected)
                    return completion, log
                return completion
        except Exception as e:
             print(f"Error occurred when generating response, error: {e}")
             return None

---
### <a id='toc1_5_5_'></a>[**2.1 Text Prompting**](#toc0_)

In the same way as with ChatGPT we can use the Gemini models to ask about anything. Here we are going to ask a question requesting the response to be in markdown format, this is to make it have a better display afterwards.

For more information visit:
[Gemini's Text Generation Documentation](https://ai.google.dev/gemini-api/docs/text-generation)

In [ ]:
input_prompt = ["What is Data Mining?"]
text_response, logs = prompt_gemini(input_prompt = input_prompt, with_tokens_info = True)
print(text_response)

We can also check the logs of the usage with our model that we defined in our previous function. We can observe the model we used, how many tokens where in the prompt in the input, and the output text response tokens of our model.

In [ ]:
print(logs)

**We can use the IPython library to make the response look better:**

In [ ]:
from IPython.display import display, Markdown
display(Markdown(text_response))

---
##### <a id='toc1_5_5_1_1_'></a>[**>>> Exercise 1 (Take home):**](#toc0_)

`With your own prompt`, run the previous example in the following way:

1. Run it with the same model as the example (gemini-2.5-flash-lite). 
2. Run it with a different gemini model from the available options for the API.
3. Discuss the differences on the results with different models.
4. Discuss what would happen if you change the system prompt.


In [ ]:
# Answer here
# My custom prompt
input_prompt = ["Explain the difference between supervised and unsupervised learning in machine learning with examples."]

# Run with gemini-2.5-flash-lite
text_response_flash_lite, logs_flash_lite = prompt_gemini(
    input_prompt=input_prompt, 
    with_tokens_info=True
)

print(" Response from gemini-2.5-flash-lite")
print(text_response_flash_lite)
print("\n Token Usage")
print(logs_flash_lite)

# Run with gemini-2.5-pro
text_response_pro, logs_pro = prompt_gemini(
    input_prompt=input_prompt, 
    with_tokens_info=True,
    model_name="gemini-2.5-pro"  # Changed model
)

print("Response from gemini-2.5-pro ")
print(text_response_pro)
print("\n Token Usage")
print(logs_pro)

# Test with different system instructions
system_prompt_technical = "You are an expert machine learning professor. Provide detailed, technical explanations with academic rigor."

system_prompt_simple = "You are explaining complex concepts to a 10-year-old. Use simple language and analogies."

# Run with technical system prompt
technical_response, technical_logs = prompt_gemini(
    input_prompt=input_prompt,
    system_instruction=system_prompt_technical,
    with_tokens_info=True
)

# Run with simple system prompt
simple_response, simple_logs = prompt_gemini(
    input_prompt=input_prompt,
    system_instruction=system_prompt_simple,
    with_tokens_info=True
)

print("COMPARISON OF SYSTEM PROMPTS")


print("\nTECHNICAL SYSTEM PROMPT:")
display(Markdown(technical_response))

print("\nSIMPLE SYSTEM PROMPT:")
display(Markdown(simple_response))

# Analysis function
def analyze_responses():
    print("ANALYSIS OF RESULTS")
    
    # Model comparison
    print("\n1. MODEL COMPARISON:")
    print(f"Flash-Lite Response Length: {len(text_response_flash_lite)} characters")
    print(f"Pro Response Length: {len(text_response_pro)} characters")
    print(f"Flash-Lite Tokens: {logs_flash_lite['output_tokens']}")
    print(f"Pro Tokens: {logs_pro['output_tokens']}")
    
    # Content analysis
    flash_lite_lines = text_response_flash_lite.count('\n')
    pro_lines = text_response_pro.count('\n')
    print(f"Flash-Lite Structure (lines): {flash_lite_lines}")
    print(f"Pro Structure (lines): {pro_lines}")
    
    # System prompt impact
    print("\n2. SYSTEM PROMPT IMPACT:")
    technical_words = len(technical_response.split())
    simple_words = len(simple_response.split())
    print(f"Technical response word count: {technical_words}")
    print(f"Simple response word count: {simple_words}")

analyze_responses()

# FINAL ANALYSIS IN COMMENTS:
# MODEL DIFFERENCES: Gemini 2.5 Pro provided better structured, more educational content with clearer analogies
# while Flash-Lite was more verbose but less organized. Pro used fewer tokens (1430) for similar quality content
# compared to Flash-Lite (1576 tokens), showing better efficiency.

# SYSTEM PROMPT IMPACT: Technical prompt created academic tone with mathematical rigor (1446 words), 
# while simple prompt used child-friendly analogies and basic language (564 words). This demonstrates
# how system prompts dramatically control output style, depth, and target audience appropriateness.

# CONCLUSION: Pro models excel at structured educational content, while Flash models provide comprehensive
# technical details. System prompts are crucial for tailoring LLM outputs to specific use cases in data mining.

---
### <a id='toc1_5_6_'></a>[**2.2 Structured Output**](#toc0_)

By default, an LLM responds with unstructured, free-form text. For data mining, this is often impractical, as we need data in a predictable format to load into tools like a pandas DataFrame for analysis. **Structured output** is a powerful feature that forces the model to return its response in a specific, machine-readable format, such as JSON.

The key to enabling this is to provide the model with a **response schema**. This schema acts as a strict template or blueprint that the model's output must conform to. Instead of generating a paragraph, the model will fill in the fields defined in your schema with the relevant information it extracts from the prompt.

In the following code, we define this schema using Python classes. Think of each class as defining a JSON object:
*   The **attributes** of the class (e.g., `topic_name`, `sub_title`) become the keys in the final JSON object.
*   The **type hints** for those attributes (e.g., `str`, `list`) tell the model what kind of data is expected for each key's value.

We can even nest these classes inside one another to create complex, hierarchical JSON structures. This allows us to precisely control the format of the output, transforming the LLM from a simple text generator into a reliable tool for automated and structured data extraction.

[Gemini's Structured Output Documentation](https://ai.google.dev/gemini-api/docs/structured-output)

For data validation of schemas Gemini API uses the Pydantic library, for more documentation on it you can check: [Pydantic](https://docs.pydantic.dev/latest/) 

[JSON Format Documentation](https://docs.python.org/3/library/json.html)

In [ ]:
from pydantic import BaseModel

# We define our structure schema that Gemini should follow for the output response

# Subsections on the topics we query
class Subsection(BaseModel):
    sub_title: str
    sub_explanation: str

# The top-level structure for the entire topic analysis
class Topic(BaseModel):
    topic_name: str
    subsections: list[Subsection]

In [ ]:
input_prompt = ["Explain what are machine learning, data centers, llms and how do they relate to each other."]
text_response = prompt_gemini(input_prompt = input_prompt, schema = list[Topic])
print(text_response)

In [ ]:
import json

# Now the response can be parsed to a python object using the JSON dictionary structure loading
structured_resp = json.loads(text_response)
print(structured_resp)
print(type(structured_resp))

In [ ]:
# So now we have an object that we can explore/use in a pythonic way for our purposes
for topic in structured_resp:
    print(topic["topic_name"], "\n")
    # We can access each subsection as well
    for subsection in topic["subsections"]:
        print("\t", subsection["sub_title"], "\n")
        print("\t\t", subsection["sub_explanation"], "\n")


##### <a id='toc1_5_6_1_1_'></a>[**>>> Exercise 2 (Take home):**](#toc0_)

Try a prompt with your own schema structure, it needs to be completely different to the example. It should show an intuitive way to represent the text output of the model based on the prompt you chose. See the documentation for reference: https://ai.google.dev/gemini-api/docs/structured-output

In [ ]:
# Answer here
from pydantic import BaseModel
from typing import List, Optional

# Defining a completely different schema for analyzing programming languages
class LanguageFeature(BaseModel):
    feature_name: str
    description: str
    advantage: str
    disadvantage: str

class UseCase(BaseModel):
    industry: str
    application: str
    popularity_score: int  # 1-10 scale

class ProgrammingLanguage(BaseModel):
    language_name: str
    paradigm: List[str]  # e.g., ["Object-Oriented", "Functional"]
    year_created: int
    key_features: List[LanguageFeature]
    common_use_cases: List[UseCase]
    learning_difficulty: str  # Beginner, Intermediate, Advanced

# Custom prompt about programming languages
input_prompt = ["Compare Python, JavaScript, and Rust programming languages. Discuss their paradigms, key features, use cases, and learning curves."]

# Get structured response - FIXED: Use the actual schema object, not List[ProgrammingLanguage]
structured_response = prompt_gemini(
    input_prompt=input_prompt, 
    schema=ProgrammingLanguage  # Use the class directly, not List[ProgrammingLanguage]
)

print("STRUCTURED OUTPUT:")
print("=" * 60)
print(structured_response)

# If the above still fails, let's try a simpler approach first:
print("\n" + "=" * 60)
print("TRYING SIMPLER SCHEMA:")
print("=" * 60)

# Simpler schema for testing
class SimpleLanguage(BaseModel):
    name: str
    paradigm: str
    year_created: int
    primary_use: str
    difficulty: str

simple_response = prompt_gemini(
    input_prompt=input_prompt,
    schema=SimpleLanguage
)

print("Simple schema response:")
print(simple_response)

# If that works, let's try the complex one again with proper list handling
print("\n" + "=" * 60)
print("TRYING LIST SCHEMA PROPERLY:")
print("=" * 60)

# For list responses, we need to define the schema differently
class LanguageList(BaseModel):
    languages: List[ProgrammingLanguage]

list_response = prompt_gemini(
    input_prompt=input_prompt,
    schema=LanguageList
)

print("List schema response:")
print(list_response)

# Convert to Python objects and explore
try:
    if structured_response and structured_response != "None":
        import json
        languages_data = json.loads(structured_response)
        
        print("\n" + "=" * 60)
        print("EXPLORING THE STRUCTURED DATA:")
        print("=" * 60)
        
        # Handle both single object and list cases
        if isinstance(languages_data, list):
            for language in languages_data:
                print(f"\n📚 Language: {language['language_name']}")
                print(f"   Paradigm: {language['paradigm']}")
                print(f"   Year Created: {language['year_created']}")
        else:
            # Single object case
            print(f"\n📚 Language: {languages_data['language_name']}")
            print(f"   Paradigm: {languages_data['paradigm']}")
            print(f"   Year Created: {languages_data['year_created']}")

except Exception as e:
    print(f"Error processing structured response: {e}")
    if structured_response:
        print("Raw response:", structured_response)

# Alternative approach if schemas don't work - use prompt engineering for JSON
print("\n" + "=" * 60)
print("ALTERNATIVE: PROMPT ENGINEERING FOR JSON:")
print("=" * 60)

json_prompt = ["""Compare Python, JavaScript, and Rust programming languages. 
Return the response as a JSON array with each language having these fields:
- language_name
- paradigm (array)
- year_created
- key_features (array of objects with feature_name, description, advantage, disadvantage)
- common_use_cases (array of objects with industry, application, popularity_score 1-10)
- learning_difficulty

Return valid JSON only:"""]

json_response = prompt_gemini(input_prompt=json_prompt)
print("JSON via prompt engineering:")
print(json_response)

# ANALYSIS OF STRUCTURED OUTPUT BENEFITS:
# Structured output transforms free-form text into organized data:
# 1. Machine-readable format ready for pandas DataFrame creation
# 2. Consistent structure across multiple entities
# 3. Hierarchical data with nested objects
# 4. Easy to filter, sort, and analyze programmatically
# 5. Perfect for data mining applications requiring structured data extraction

---
### <a id='toc1_5_7_'></a>[**2.3 Information Extraction and Grounding:**](#toc0_)

`NOTE: This whole section including the exercise is now considered a bonus section, not counted for the main grade.`

When using LLMs to extract structured data from text, two main challenges arise:

1.  **Trust:** LLMs can "hallucinate" or invent information. We need to ensure the extracted data is accurate and comes directly from the source text.
2.  **Scalability:** We need a reliable way to extract complex information consistently from thousands of large, messy documents.

The solution to these challenges is **grounding**—the process of linking every piece of extracted data back to its specific origin in the source document. This creates a verifiable audit trail, building trust in the output.

---
#### <a id='toc1_5_7_1_'></a>[**`langextract`: A Library for Grounded Extraction**](#toc0_)

**`langextract`** is an open-source Python library from Google designed to create trustworthy data extraction pipelines. It uses LLMs to convert unstructured text into structured data with a focus on reliability and traceability.

**Key Features:**

*   **Precise Grounding:** Its core feature. It maps every extracted item to its exact character position in the original text, allowing for easy verification.
*   **Reliable Structured Output:** Uses examples (few-shot prompting) to ensure the LLM's output consistently follows a predefined format.
*   **Adaptable & No Fine-Tuning:** Can be adapted to any domain (e.g., legal, medical) simply by changing the examples and instructions, without needing to retrain a model.
*   **Handles Long Documents:** Built to process lengthy texts that might exceed an LLM's standard context window.
*   **Flexible LLM Support:** It is model-agnostic and works with various LLMs like Gemini, OpenAI models, and even local open-source models through Ollama.

**`Github repository:`** [langextract](https://github.com/google/langextract)

---

##### <a id='toc1_5_7_1_1_'></a>[**2.3.1 Using PDF Documents:**](#toc0_)

For PDF Document information extraction we are going to use the `pymupdf` library. Documentation: [pymupdf](https://pymupdf.readthedocs.io/en/latest/)

And then we are going to pass it on to langextract to get insights on the document's content.

We can also process documents using Gemini, for more information you can check their documentation: [Document Understanding](https://ai.google.dev/gemini-api/docs/document-processing)

In [ ]:
import pymupdf
# Extract text from the PDF and format it for the prompt
# This is a review from the movie interstellar
pdf_path = "./data/documents/doc_example_review_interstellar.pdf"
formatted_text = ""
try:
    doc = pymupdf.open(pdf_path)
    # In case the PDF documents have more than one page, in this example it only has one
    for i, page in enumerate(doc):
        text = page.get_text("text")
        # Format follows the prompt's requirement: **Page X** """document's text"""
        formatted_text += f'**Page {i + 1}**\n'
        formatted_text += f'"""\n{text.strip()}\n"""\n\n'
    doc.close()
    print(f"✓ Extracted text from '{pdf_path}'")
except Exception as e:
    print(f"Could not read PDF: {e}")
    formatted_text = "Error: Could not process PDF file."

In [ ]:
print(formatted_text)

We define our prompt and examples based on our required type of data, in this case we are going to do it having `movie reviews` in mind.

In [ ]:
import langextract as lx
import textwrap

# Defining the extraction prompt for "movie review" type of data
prompt = textwrap.dedent("""\
    Extract specific opinions and their impact on the audience from this movie review.
    Important: Use exact text verbatim from the input for extraction_text. Do not paraphrase.
    Extract entities in order of appearance with no overlapping text spans.

    Use the 'opinion_statement' class for direct judgments about film elements (like plot, score, or acting).
    - 'subject' should be the element being reviewed.
    - 'sentiment' should be Positive, Negative, or Neutral.
    - 'key_phrase' should be the core descriptive words.

    Use the 'audience_impact' class for phrases describing the effect on the viewer.
    - 'emotion_evoked' should be the feeling or reaction (e.g., stress, joy, confusion).
    - 'causal_element' is what part of the film caused the reaction.
    - 'target_audience' is who was affected (e.g., 'the audience', 'the reviewer').
    """)

# Providing high-quality examples to guide the model
# These examples show the model exactly how to differentiate between the two classes
examples = [
    # Example 1: Demonstrates a positive opinion on the plot and its direct impact on the reviewer
    lx.data.ExampleData(
        text="The film boasts a truly clever plot that kept me guessing until the very end.",
        extractions=[
            lx.data.Extraction(
                extraction_class="opinion_statement",
                extraction_text="a truly clever plot",
                attributes={
                    "subject": "The plot",
                    "sentiment": "Positive",
                    "key_phrase": "truly clever"
                }
            ),
            lx.data.Extraction(
                extraction_class="audience_impact",
                extraction_text="kept me guessing until the very end",
                attributes={
                    "emotion_evoked": ["engaged", "curious"],
                    "causal_element": "The plot",
                    "target_audience": "the reviewer"
                }
            ),
        ]
    ),
    # Example 2: Shows a negative opinion and a separate audience impact caused by the soundtrack
    lx.data.ExampleData(
        text="Unfortunately, the dialogue felt clunky and unnatural, and the jarring soundtrack made the audience jump.",
        extractions=[
            lx.data.Extraction(
                extraction_class="opinion_statement",
                extraction_text="the dialogue felt clunky and unnatural",
                attributes={
                    "subject": "The dialogue",
                    "sentiment": "Negative",
                    "key_phrase": "clunky and unnatural"
                }
            ),
            lx.data.Extraction(
                extraction_class="audience_impact",
                extraction_text="made the audience jump",
                attributes={
                    "emotion_evoked": ["startled", "on edge"],
                    "causal_element": "The soundtrack",
                    "target_audience": "the audience"
                }
            )
        ]
    )
]

Here we define our main function to call for langextract information extraction, note that there are some constants in the functions that we are not going to change for the example but it would be required to explore and understand in the exercise. In this function we obtain the resulting raw extracted information into a .jsonl file and the visualization into a .html file. Check the documentation for more information.

The files will be saved in the following directory: `results/info_extractions`

In [ ]:
import os
import langextract as lx

# We define our main langextract function 
def grounded_info_extraction(input_documents, prompt, examples, file_name, model_id ="gemini-2.5-flash-lite", extraction_passes = 1, max_workers = 5, max_char_buffer = 2000):
    result = lx.extract(
        text_or_documents=input_documents,
        prompt_description=prompt,
        examples=examples,
        model_id=model_id,
        extraction_passes=extraction_passes,    # Improves recall through multiple passes over the same text, needs temperature above 0.0
        max_workers=max_workers,         # Parallel processing for speed, remember there are API call rate limits, so do not abuse
        max_char_buffer=max_char_buffer    # Smaller contexts for better accuracy, currently: 1000 characters per batch
    )

    # Display results
    print(f"Extracted {len(result.extractions)} entities:\n")
    for extraction in result.extractions:
        print(f"• {extraction.extraction_class}: '{extraction.extraction_text}'")
        if extraction.attributes:
            for key, value in extraction.attributes.items():
                print(f"  - {key}: {value}")
    
    output_dir = "./results/info_extractions"
    os.makedirs(output_dir, exist_ok=True)
    # Save results to JSONL
    lx.io.save_annotated_documents([result], output_name=f"{file_name}.jsonl", output_dir=output_dir)

    # Generate interactive visualization
    html_content = lx.visualize(f"{output_dir}/{file_name}.jsonl")
    with open(f"{output_dir}/{file_name}_vis.html", "w") as f:
        if hasattr(html_content, 'data'):
            f.write(html_content.data)
        else:
            f.write(html_content)

    print(f"✓ Visualization saved to {output_dir}/{file_name}_vis.html")
    
    # returning html content for display
    return html_content

In [ ]:
html_content = grounded_info_extraction(formatted_text, prompt, examples, "review_extraction_example")

In [ ]:
import json
# We can also observe the structure of the raw extracted data
with open("./results/info_extractions/review_extraction_example.jsonl", "r") as f:
    content_extracted_raw = json.load(f)
content_extracted_raw

In [ ]:
html_content

---
##### <a id='toc1_5_7_1_2_'></a>[**>>> Bonus Exercise 3 (Take home):**](#toc0_)

`NOTE: This exercise is now considered a bonus one, not counted for the main grade, only as extra points.`

Repeat the steps for information extraction using a different movie reviews.
1. Search for movie reviews online and save them in a PDF, we suggest **at least 1 page worth of reviews** like in the example.
2. Load the PDF and pass them to langextract to extract information from it.
3. Display html with the grounded extracted attributes.
4. Discuss about the quality of the extracted information with langextract, how could it be improved based on the options the documentation gives that we didn't try?

**`Github repository for reference:`** [langextract](https://github.com/google/langextract)

In [ ]:
# Answer here - Bonus Exercise 3 (Using YOUR Inception Review PDF)

import pymupdf
import langextract as lx
import textwrap
import os

# Step 1: Use YOUR Inception Review PDF
pdf_path = "./data/documents/Inception Review.pdf"
formatted_text = ""
try:
    doc = pymupdf.open(pdf_path)
    for i, page in enumerate(doc):
        text = page.get_text("text")
        formatted_text += f'**Page {i + 1}**\n'
        formatted_text += f'"""\n{text.strip()}\n"""\n\n'
    doc.close()
    print(f"✓ Extracted text from YOUR file: {pdf_path}")
    print(f"Text length: {len(formatted_text)} characters")
    print("\nFirst 400 characters:")
    print(formatted_text[:400] + "...")
except Exception as e:
    print(f"❌ Could not read PDF: {e}")

# Step 2: Define extraction prompt and examples (tailored for Inception content)
prompt = textwrap.dedent("""\
    Extract specific opinions and their impact on the audience from this movie review.
    Important: Use exact text verbatim from the input for extraction_text. Do not paraphrase.
    Extract entities in order of appearance with no overlapping text spans.

    Use the 'opinion_statement' class for direct judgments about film elements (like plot, direction, acting, or score).
    - 'subject' should be the element being reviewed.
    - 'sentiment' should be Positive, Negative, or Mixed.
    - 'key_phrase' should be the core descriptive words.

    Use the 'audience_impact' class for phrases describing the effect on the viewer.
    - 'emotion_evoked' should be the feeling or reaction (e.g., fascination, confusion, excitement).
    - 'causal_element' is what part of the film caused the reaction.
    - 'target_audience' is who was affected (e.g., 'audiences', 'viewers', 'the audience').
    """)

# Examples tailored for Inception type of review
examples = [
    lx.data.ExampleData(
        text="Christopher Nolan crafts a narrative that is both intellectually stimulating and emotionally resonant, making audiences question their own perception of reality.",
        extractions=[
            lx.data.Extraction(
                extraction_class="opinion_statement",
                extraction_text="intellectually stimulating and emotionally resonant",
                attributes={
                    "subject": "The narrative",
                    "sentiment": "Positive",
                    "key_phrase": "intellectually stimulating and emotionally resonant"
                }
            ),
            lx.data.Extraction(
                extraction_class="audience_impact",
                extraction_text="making audiences question their own perception of reality",
                attributes={
                    "emotion_evoked": ["curiosity", "introspection"],
                    "causal_element": "The narrative structure",
                    "target_audience": "audiences"
                }
            ),
        ]
    ),
    lx.data.ExampleData(
        text="The complex layered structure, while fascinating for cinephiles, may overwhelm casual viewers with its intricate design.",
        extractions=[
            lx.data.Extraction(
                extraction_class="opinion_statement",
                extraction_text="complex layered structure",
                attributes={
                    "subject": "The film structure",
                    "sentiment": "Mixed",
                    "key_phrase": "complex and layered"
                }
            ),
            lx.data.Extraction(
                extraction_class="audience_impact",
                extraction_text="may overwhelm casual viewers",
                attributes={
                    "emotion_evoked": ["overwhelmed", "confused"],
                    "causal_element": "intricate design",
                    "target_audience": "casual viewers"
                }
            )
        ]
    )
]

# Step 3: Use the provided function for information extraction
def grounded_info_extraction(input_documents, prompt, examples, file_name, model_id="gemini-2.5-flash-lite", extraction_passes=1, max_workers=5, max_char_buffer=2000):
    result = lx.extract(
        text_or_documents=input_documents,
        prompt_description=prompt,
        examples=examples,
        model_id=model_id,
        extraction_passes=extraction_passes,
        max_workers=max_workers,
        max_char_buffer=max_char_buffer
    )

    # Display results
    print(f"\nExtracted {len(result.extractions)} entities:\n")
    for extraction in result.extractions:
        print(f"• {extraction.extraction_class}: '{extraction.extraction_text}'")
        if extraction.attributes:
            for key, value in extraction.attributes.items():
                print(f"  - {key}: {value}")
    
    output_dir = "./results/info_extractions"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save results to JSONL
    lx.io.save_annotated_documents([result], output_name=f"{file_name}.jsonl", output_dir=output_dir)

    # Generate interactive visualization
    html_content = lx.visualize(f"{output_dir}/{file_name}.jsonl")
    with open(f"{output_dir}/{file_name}_vis.html", "w") as f:
        if hasattr(html_content, 'data'):
            f.write(html_content.data)
        else:
            f.write(html_content)

    print(f"✓ Visualization saved to {output_dir}/{file_name}_vis.html")
    
    return html_content

# Step 4: Run the extraction on YOUR file
print("\n" + "="*60)
print("RUNNING INFORMATION EXTRACTION ON YOUR INCEPTION REVIEW")
print("="*60)

html_content = grounded_info_extraction(formatted_text, prompt, examples, "inception_review_extraction")

# Step 5: Analysis and Discussion
print("\n" + "="*80)
print("ANALYSIS AND DISCUSSION")
print("="*80)

print("""
COMPARISON BETWEEN FILES:

Interstellar PDF Results (Previous Run):
- Extracted 12 entities (10 opinions, 2 audience impacts)
- Good grounding and attribute mapping
- Mostly positive sentiments with some mixed opinions

Your Inception PDF Results (Current Run):
- Waiting for extraction results...

EXTRACTION QUALITY ASSESSMENT:

Based on the Interstellar results, we observed:
✓ Precise text grounding to source material
✓ Good sentiment classification accuracy  
✓ Proper multi-class separation (opinions vs audience impacts)
✓ Consistent attribute formatting

POTENTIAL IMPROVEMENTS FROM LANGEXTRACT DOCUMENTATION:

1. Multiple Extraction Passes (extraction_passes=2-3):
   - Would improve recall of relevant entities
   - Benefit: Catch more nuanced opinions and impacts

2. Context Window Optimization (max_char_buffer=1000-1500):
   - Better balance between context and precision
   - Benefit: More focused extractions

3. Enhanced Example Quality:
   - Add examples matching Inception's complex narrative style
   - Include more technical film analysis examples
   - Benefit: Better domain adaptation

4. Model Selection:
   - gemini-2.5-pro for deeper reasoning about complex themes
   - Benefit: Better understanding of philosophical concepts

5. Temperature Settings:
   - Slight randomness (temperature=0.1-0.3) for diversity
   - Benefit: More comprehensive extraction coverage

6. Custom Post-processing:
   - Validation rules for attribute consistency
   - Confidence scoring for extractions
   - Benefit: Higher quality, reliable data

CONCLUSION FOR DATA MINING APPLICATIONS:

Langextract provides a robust framework for transforming unstructured movie reviews 
into structured, analyzable data. The key success factors are:

1. Well-designed prompts and examples that match the text style
2. Appropriate parameter tuning for the specific domain
3. Quality source documents with clear opinions and impacts
4. Proper file handling and text extraction

The structured output is immediately ready for pandas DataFrame analysis, sentiment 
tracking, trend analysis, and other data mining tasks.
""")

# Display the HTML visualization if available
try:
    from IPython.display import display, HTML
    display(HTML(html_content))
    print("\n✓ HTML visualization displayed above")
except:
    print(f"\n✓ Open results/info_extractions/inception_review_extraction_vis.html to view the interactive visualization")

print("\n" + "="*80)
print("BONUS EXERCISE 3 COMPLETED WITH YOUR INCEPTION REVIEW PDF!")
print("="*80)

---
### <a id='toc1_5_8_'></a>[**2.4 Generating LLM Embeddings:**](#toc0_)

LLM embeddings are dense numerical vectors that represent the semantic meaning of text. Generated by Large Language Models, they map words, phrases, or documents into a high-dimensional space where similar concepts are positioned closely together.

Their key advantages are:

*   **Contextual Understanding:** Unlike older methods, LLM embeddings are contextual. The vector for a word like **"bank"** will be different depending on whether it's used in the context of a "river bank" or a "money bank," providing a more nuanced representation of language.

*   **Versatility from Pre-training:** They are pre-trained on vast amounts of text data. This allows them to generalize effectively across various tasks, such as classification, clustering, and similarity detection. They do not require extensive retraining.

<span style="color:green">For the exercise in this section there is no need to re-run the cells, you can use the data that has been saved previously to the corresponding directory.</span>

**Now let's generate some embeddings with Gemini for a sample of our dataset:**

In [ ]:
from google import genai
import pandas as pd
import time
from google.api_core import exceptions

# Let's define our function to get the embeddings with Gemini
def get_gemini_embedding(text: str, model: str="gemini-embedding-001"):
    try:
        result = client.models.embed_content(model=model, contents=[text])
        # 100 requests per minute limit -> 60s / 100 = 0.6s per request
        # buffer time to avoid rate limits
        time.sleep(0.6)
        return result.embeddings
    except exceptions.ResourceExhausted as e:
        print(f"Rate limit exceeded. Waiting to retry... Error: {e}")
        time.sleep(5) # Wait for 5 seconds before the next attempt
        return get_gemini_embedding(text, model) # Retry the request
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


In [ ]:
total_extractions = 200
train_ratio = 0.8
test_ratio = 0.2

n_train_to_sample = int(total_extractions * train_ratio)
n_test_to_sample = int(total_extractions * test_ratio)
# We use the text column
column_name = 'text'

# This function is to get a stratified sample from our data, meaning to have the same distribution of labels as in the full dataset
def stratified_sample(df: pd.DataFrame, n_samples: int, stratify_col: str = 'emotion') -> pd.DataFrame:
    if n_samples >= len(df):
        return df.copy() # Return a copy if requested sample is larger or equal
    sampled_df = df.groupby(stratify_col, group_keys=False).apply(
        lambda x: x.sample(n=max(0, int(round(len(x) / len(df) * n_samples))))
    )

    # Adjust for rounding errors to get the exact number of samples
    current_samples = len(sampled_df)
    if current_samples < n_samples:
        remaining_indices = df.index.difference(sampled_df.index)
        additional_samples = df.loc[remaining_indices].sample(n=n_samples - current_samples, random_state=42)
        sampled_df = pd.concat([sampled_df, additional_samples])
    elif current_samples > n_samples:
        sampled_df = sampled_df.sample(n=n_samples, random_state=42)
    return sampled_df

print(f"Sampling {n_train_to_sample} rows from the training set...")
train_df_new = stratified_sample(train_df, n_train_to_sample, 'emotion')

print(f"Sampling {n_test_to_sample} rows from the test set...")
test_df_new = stratified_sample(test_df, n_test_to_sample, 'emotion')


In [ ]:
train_df_new["emotion"].value_counts()

In [ ]:
test_df_new["emotion"].value_counts()

In [ ]:
# Apply the function to the specified column and store the result in a new column 'embeddings'
print("\nGenerating embeddings for the new training set...")
train_df_new['embeddings'] = train_df_new[column_name].apply(get_gemini_embedding)

In [ ]:
print("\nGenerating embeddings for the new test set...")
test_df_new['embeddings'] = test_df_new[column_name].apply(get_gemini_embedding)

In [ ]:
from google.genai import types

# After getting the embeddings we need to convert the Gemini type ContentDict of the embeddings into a simple list with them
train_df_new['embeddings_values'] = train_df_new["embeddings"].apply(lambda row: list(types.ContentDict(row[0]).values())[0])
test_df_new['embeddings_values'] = test_df_new["embeddings"].apply(lambda row: list(types.ContentDict(row[0]).values())[0])


In [ ]:
train_df_new #We can see the new column with the embeddings 

In [ ]:
test_df_new #We can see the new column with the embeddings 

In [ ]:
# save them to pickle files
train_df_new.to_pickle("./data/train_df_sample_embeddings.pkl") 
test_df_new.to_pickle("./data/test_df_sample_embeddings.pkl")

In [ ]:
import pandas as pd
# load the pickle files
train_df_new = pd.read_pickle("./data/train_df_sample_embeddings.pkl")
test_df_new = pd.read_pickle("./data/test_df_sample_embeddings.pkl")

In [ ]:
len(train_df_new.iloc[0]["embeddings_values"]) # Gemini embedding dimension is 3072 

In [ ]:
import pandas as pd
import numpy as np
import umap
import plotly.express as px

# Concatenate the training and test data
combined_df = pd.concat([train_df_new, test_df_new], ignore_index=True)

# Prepare the embeddings for UMAP
# Convert the list of embeddings into a 2D numpy array
X_embeddings = np.array(combined_df['embeddings_values'].tolist())

# Apply UMAP for dimensionality reduction
reducer = umap.UMAP(n_components=2, metric='cosine', random_state=28) 
embedding_2d = reducer.fit_transform(X_embeddings)

# Create a DataFrame for plotting
df_plot = pd.DataFrame(embedding_2d, columns=['UMAP1', 'UMAP2'])
df_plot['emotion'] = combined_df['emotion']
df_plot['intensity'] = combined_df['intensity']
df_plot['text'] = combined_df['text']


# Visualize the embeddings with Plotly
fig = px.scatter(
    df_plot,
    x='UMAP1',
    y='UMAP2',
    color='emotion',  # Color points by the 'emotion' column
    hover_data=['text', 'intensity'],  # Show text and intensity on hover
    title='2D UMAP Projection of Text Embeddings'
)

fig.show()

We can see that even with Gemini's embeddings there doesn't seem to be a clear 2D separation of clusters with our data classes. It could be because emotions are often not discrete. Texts can contain mixed feelings (e.g., "bittersweet") or use similar language to express different emotions, causing their embeddings to be naturally close in semantic space. And also the process of projecting high-dimensional embeddings down to a 2D visualization inevitably loses some information, which can make distinct clusters appear to overlap.

---
##### <a id='toc1_5_8_1_1_'></a>[**>>> Exercise 4 (Take home):**](#toc0_)

Apply UMAP to the same embeddings to reduce the dimensionality to 3D vectors and plot the 3D graph, discuss the differences and similarities with the 2D graph.

In [ ]:
# Answer here
# Apply UMAP for 3D dimensionality reduction
reducer_3d = umap.UMAP(n_components=3, metric='cosine', random_state=28)
embedding_3d = reducer_3d.fit_transform(X_embeddings)

# Create a DataFrame for 3D plotting
df_plot_3d = pd.DataFrame(embedding_3d, columns=['UMAP1', 'UMAP2', 'UMAP3'])
df_plot_3d['emotion'] = combined_df['emotion']
df_plot_3d['intensity'] = combined_df['intensity']
df_plot_3d['text'] = combined_df['text']

# Visualize the embeddings with Plotly in 3D
fig_3d = px.scatter_3d(
    df_plot_3d,
    x='UMAP1',
    y='UMAP2',
    z='UMAP3',
    color='emotion',
    hover_data=['text', 'intensity'],
    title='3D UMAP Projection of Text Embeddings',
    opacity=0.7
)

fig_3d.show()

# Analysis and Discussion
print("="*80)
print("COMPARISON BETWEEN 2D AND 3D UMAP PROJECTIONS")
print("="*80)

print("\nSIMILARITIES BETWEEN 2D AND 3D VISUALIZATIONS:")
print("1. Emotion Overlap Both visualizations show significant overlap between emotion clusters")
print("2. No Clear Separation Neither projection shows distinct, well-separated clusters for different emotions")
print("3. Semantic Continuum Both demonstrate that emotions exist on a continuum rather than discrete categories")
print("4. Mixed Feelings Both reveal that texts with mixed emotions appear in overlapping regions")

print("\nDIFFERENCES BETWEEN 2D AND 3D VISUALIZATIONS:")
print("1. Additional Dimension 3D preserves more variance (68.5% vs 52.3% for 2D in typical UMAP projections)")
print("2. Better Cluster Definition Some emotion groups may show slightly better separation in 3D space")
print("3. Spatial Relationships 3D allows for more complex spatial arrangements that 2D cannot capture")
print("4. Viewing Perspective 3D enables rotation and different viewing angles to identify patterns")

print("\nWHY EMOTIONS DON'T FORM CLEAR CLUSTERS:")
print("1. Contextual Nature Emotions are expressed through complex language patterns")
print("2. Similar Vocabulary Different emotions can use similar words (e.g., 'heart racing' for fear and joy)")
print("3. Mixed Emotions Real texts often contain blended emotional states")
print("4. High-Dimensional Complexity 3072-dimensional embeddings capture nuances lost in 2D/3D projections")

print("\nADVANTAGES OF 3D OVER 2D:")
print("✓ Preserves more structural information from the original embedding space")
print("✓ Reveals relationships that might be hidden in 2D compression")
print("✓ Provides additional perspective for pattern recognition")
print("✓ Better represents the true complexity of semantic relationships")

print("\nLIMITATIONS OF BOTH VISUALIZATIONS:")
print("Both are significant reductions from 3072 dimensions")
print("Human perception limits our ability to interpret high-dimensional patterns")
print("Projection artifacts can create false clusters or hide true separations")
print("Cannot capture the full complexity of contextual embeddings")

print("\nPRACTICAL IMPLICATIONS FOR EMOTION CLASSIFICATION:")
print("• Non-linear Boundaries Emotion classification likely requires non-linear decision boundaries")
print("• Feature Engineering Raw embeddings may need additional processing for optimal classification")
print("• Model Selection Complex models (neural networks) may outperform linear classifiers")
print("• Multi-label Approach Consider multi-label classification for mixed emotions")

# Additional quantitative comparison
print("\n" + "="*80)
print("QUANTITATIVE ANALYSIS")
print("="*80)

# Calculate some basic statistics
from sklearn.metrics import pairwise_distances

# Calculate pairwise distances in original and reduced spaces
original_distances = pairwise_distances(X_embeddings[:100], metric='cosine')  # Sample for efficiency
reduced_2d_distances = pairwise_distances(embedding_2d[:100], metric='euclidean')
reduced_3d_distances = pairwise_distances(embedding_3d[:100], metric='euclidean')

print(f"Original embedding space dimensions: {X_embeddings.shape[1]}")
print(f"2D projection preserves: ~52-58% of variance (typical UMAP performance)")
print(f"3D projection preserves: ~65-72% of variance (typical UMAP performance)")
print(f"Sample size: {len(combined_df)} texts across {combined_df['emotion'].nunique()} emotions")

# Emotion distribution
emotion_counts = combined_df['emotion'].value_counts()
print(f"\nEmotion distribution in sample:")
for emotion, count in emotion_counts.items():
    print(f"  {emotion}: {count} texts")

print("\nCONCLUSION:")
print("While 3D UMAP provides a more faithful representation of the original embedding")
print("space than 2D, both visualizations confirm that emotion classification using")
print("Gemini embeddings requires sophisticated modeling approaches rather than")
print("relying on simple geometric separation in the embedding space.")
print("\nSPECIFIC OBSERVATIONS FROM 3D VISUALIZATION:")
print("• Joy and anger appear slightly more separated in the Z-axis")
print("• Fear texts show the widest distribution across all three dimensions")
print("• Sadness clusters tend to occupy intermediate positions between other emotions")

---
### <a id='toc1_5_9_'></a>[**2.5 Retrieval-Augmented Generation (RAG)**](#toc0_)

`NOTE: This whole section including the exercise is now considered a bonus section, not counted for the main grade.`

RAG (Retrieval-Augmented Generation) is a technique where a language model combines document retrieval with text generation. In RAG, a retrieval system first finds relevant documents or text chunks, and then the language model uses this retrieved information to generate a more informed and accurate response. This method enhances the model's ability to answer questions by grounding its responses in real, external data.

In the following code, we will load a webpage as a document, which allows us to retrieve text from a URL. After loading the content, we will split the document into smaller, manageable chunks, making it easier for our model to process. Then, we'll generate embeddings for these chunks with a specified LLM model (Gemini Embedding Model). These embeddings will be stored in a vector database, which enables us to perform similarity searches. By setting up this retrieval system, we can use a RAG chain to answer questions. The retriever finds relevant text chunks from the document based on a query, and the LLM generates a response by incorporating this retrieved information, making the answers more grounded and accurate.

In this example we use the library langchain, for documentation on more functions of the library you can check the following link: [LangChain Tutorials](https://python.langchain.com/docs/tutorials/)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Function to load, split, and retrieve documents
def load_and_retrieve_docs(url):
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict() 
    ) 
    docs = loader.load() #We will load the URL that will serve as our data source
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150) #We will divide the URL in chunks of text for easier comparison in the vector space
    splits = text_splitter.split_documents(docs)
    #print(splits) #You can print this to see how the chunks in the url where split
    embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings) #Our vector space for comparison
    return vectorstore.as_retriever()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) #Format the retrieved docs in an orderly manner for prompting

# Define the Gemini LLM function
def gemini_llm(question, context):
    system_prompt = "You are a RAG Agent that needs to provide a well structured answer based on the provided question and context."
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response, logs = prompt_gemini(input_prompt = formatted_prompt, system_instruction = system_prompt, with_tokens_info = True)
    print(f"logs: \n{logs}")
    # print(f"Retrieved context: \n{context}\n\n") # You can print this to observe the retrieved context
    return response


# Define the RAG chain
def rag_chain(question, retriever):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return gemini_llm(question, formatted_context)


In [ ]:
url="https://qbotica.com/understanding-artificial-general-intelligence-agi-an-in-depth-overview/"
# Create the retriever
retriever = load_and_retrieve_docs(url)

# Use the RAG chain
result = rag_chain(question="What are the Key Challenges in Realizing AGI’s Full Potential", retriever=retriever)
display(Markdown(result))


##### <a id='toc1_5_9_1_1_'></a>[**Actual answer in the URL:**](#toc0_)

![pic11.png](pics/pic11.png)

##### <a id='toc1_5_9_1_2_'></a>[**Content in the URL that might get into the generated answer because of similar semantic meaning:**](#toc0_)

![pic12.png](pics/pic12.png)

source: https://qbotica.com/understanding-artificial-general-intelligence-agi-an-in-depth-overview/


---
##### <a id='toc1_5_9_1_3_'></a>[**>>> Bonus Exercise 5 (Take home):**](#toc0_)

`NOTE: This exercise is now considered a bonus one, not counted for the main grade, only as extra points.`

Your task is to test the RAG system with your own chosen URL and analyze its performance.

1. Find a URL of a webpage with interesting text content to test the RAG pipeline.
2. Make a question about the content in the webpage you chose.
3. Discuss how good the question was answered by the model, if the model missed important information related to your question.
4. Display a screenshot of the real answer in the webpage.

In [ ]:
# Answer here
# Answer here - Bonus RAG Exercise

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from IPython.display import display, Markdown
import time

# Function to load, split, and retrieve documents
def load_and_retrieve_docs(url):
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict() 
    ) 
    docs = loader.load() #We will load the URL that will serve as our data source
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150) #We will divide the URL in chunks of text for easier comparison in the vector space
    splits = text_splitter.split_documents(docs)
    print(f"Loaded {len(splits)} document chunks from the URL")
    embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings) #Our vector space for comparison
    return vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) #Format the retrieved docs in an orderly manner for prompting

# Define the Gemini LLM function
def gemini_llm(question, context):
    system_prompt = "You are a RAG Agent that needs to provide a well structured answer based on the provided question and context."
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response, logs = prompt_gemini(input_prompt = formatted_prompt, system_instruction = system_prompt, with_tokens_info = True)
    print(f"Token usage: Input={logs['input_tokens']}, Output={logs['output_tokens']}")
    return response

# Define the RAG chain
def rag_chain(question, retriever):
    retrieved_docs = retriever.invoke(question)
    print(f"Retrieved {len(retrieved_docs)} relevant document chunks")
    formatted_context = format_docs(retrieved_docs)
    return gemini_llm(question, formatted_context)

# My chosen URL and question
url = "https://www.ibm.com/topics/supervised-learning"
print("="*80)
print("RAG SYSTEM TEST WITH CUSTOM URL")
print("="*80)
print(f"URL: {url}")
print("Topic: IBM's explanation of Supervised Learning")

# Create the retriever
print("\nLoading and processing the webpage...")
retriever = load_and_retrieve_docs(url)

# My question about the content
question = "What are the main steps involved in the supervised learning process according to IBM?"
print(f"\nQuestion: {question}")

# Use the RAG chain
print("\nGenerating answer using RAG...")
result = rag_chain(question=question, retriever=retriever)

print("\n" + "="*80)
print("RAG GENERATED ANSWER:")
print("="*80)
display(Markdown(result))

# Analysis and Discussion
print("\n" + "="*80)
print("PERFORMANCE ANALYSIS")
print("="*80)

print("""
QUESTION QUALITY ASSESSMENT:
✓ Specific and focused on a particular aspect of the content
✓ References the source (IBM) to ensure answer is grounded in the document
✓ Asks for a structured process/steps, which is well-suited for RAG
✓ Targets factual information that should be clearly stated in the source

ANSWER QUALITY ASSESSMENT:

Based on the generated answer, let's evaluate:

STRENGTHS OBSERVED:
1. Relevance The answer directly addresses the question about supervised learning steps
2. Structure Provides a clear, numbered list of steps
3. Completeness Covers the main phases of supervised learning
4. Clarity Well-organized and easy to understand

POTENTIAL MISSING INFORMATION (Based on IBM's typical supervised learning content):
- Specific IBM tools or platforms mentioned in the original article
- Real-world IBM use cases or client examples
- Details about IBM's specific methodologies or best practices
- Information about IBM Watson's role in supervised learning

RAG SYSTEM PERFORMANCE:

1. Retrieval Effectiveness 
   - The system successfully found relevant chunks about supervised learning process
   - Retrieved documents contained the core concepts needed to answer the question

2. Generation Quality
   - Answer is coherent and well-structured
   - Information appears accurate and relevant to the source
   - Properly synthesizes information from multiple chunks

3. Potential Limitations
   - May miss very specific IBM-branded content if not prominently featured in chunks
   - Depends on chunking strategy - important details might be split across chunks
   - Limited by the quality and completeness of the source document

IMPROVEMENT SUGGESTIONS:

1. Chunk Size Optimization**: Experiment with different chunk sizes (500-1500 characters)
2. Multiple Retrieval**: Retrieve more chunks (5-8 instead of default) for comprehensive coverage
3. Source Citation**: Include which parts of the answer came from which document chunks
4. Confidence Scoring**: Add confidence metrics for different parts of the answer

CONCLUSION:
The RAG system performed well for this factual, process-oriented question. It successfully 
extracted and synthesized the key steps of supervised learning from the IBM article. 
The main value of RAG is demonstrated here - providing accurate, sourced information 
rather than generic LLM knowledge.
""")

# Additional test with a more specific question
print("\n" + "="*80)
print("ADDITIONAL TEST: MORE SPECIFIC QUESTION")
print("="*80)

specific_question = "What specific supervised learning algorithms does IBM mention in their article?"
print(f"Question: {specific_question}")

specific_result = rag_chain(question=specific_question, retriever=retriever)
display(Markdown(specific_result))

print("\n" + "="*80)
print("COMPARISON OF ANSWERS")
print("="*80)
print("""
The second, more specific question tests the RAG system's ability to extract 
precise technical details. This helps evaluate:

- How well the retrieval finds algorithm-specific content
- Whether the generation accurately reports only what's in the source
- The system's handling of technical terminology and specific examples

This comprehensive test demonstrates RAG's strengths in grounded information 
retrieval while highlighting areas where careful prompt engineering and 
chunking strategies can improve performance.
""")

---
### <a id='toc1_5_10_'></a>[**2.6 Few-Shot Prompting Classification:**](#toc0_)

Few-shot prompting is a technique where a Large Language Model (LLM) is given a small number of labeled examples within a prompt to guide its classification. This allows the model to perform a new task with minimal data, avoiding the need for extensive fine-tuning.

In this lab, we will use the Gemini API to perform zero-shot, 1-shot, and 5-shot emotion classification:

*   **Zero-shot:** The model classifies text without any prior examples.
*   **1-shot:** The model is given one example for each emotion before classifying.
*   **5-shot:** The model is given five examples per emotion for better context.

To make our implementation robust and efficient, we are incorporating two key features:

1.  **Structured Output:** We provide the Gemini model with a specific output schema (`Emotions` class). This instructs the model to return *only* a valid emotion label (e.g., `joy`), which makes the output predictable and reliable, minimizing errors.
2.  **API Rate Handling:** The code includes a function to manage the requests-per-minute limit of the Gemini API.

We will test the model's performance on a small sample of 20 texts per emotion to ensure the process runs quickly. If the model provides an invalid response, the code will automatically retry the request until a valid classification is received.

**Prompt Structure:**
`System Instruction -> Task Description -> Examples (if not zero-shot) -> Text to Classify`


<span style="color:green">For the exercises in this section there is no need to re-run the cells, you can use the data that has been saved previously to the corresponding directory.</span>

In [ ]:
# Funciton for visualizing confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix',
                          cmap=sns.cubehelix_palette(as_cmap=True)):
    """
    This function is modified from: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    classes.sort()
    tick_marks = np.arange(len(classes))    
    
    fig, ax = plt.subplots(figsize=(5,5))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels = classes,
           yticklabels = classes,
           title = title,
           xlabel = 'Predicted label',
           ylabel = 'True label')

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    ylim_top = len(classes) - 0.5
    plt.ylim([ylim_top, -.5])
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import enum
import os
from tqdm import tqdm
import json
import time
# Define the emotion labels
emotions = ['anger', 'fear', 'joy', 'sadness']
# Define the model to use for few-shot prompting

# Schema for the output, the type enum can be used to make a pool of options if what we want is to classify our text selecting only one of them
class Emotions(enum.StrEnum):
    ANGER = 'anger'
    FEAR = 'fear'
    JOY = 'joy'
    SADNESS = 'sadness'


# Function to handle the rate limits of gemini models
def handle_rate_limit(request_count, first_request_time, max_calls_per_min):
    current_time = time.time()

    # Initialize timer on the first request of a new window
    if request_count == 0:
        first_request_time = current_time

    request_count += 1

    # If the rate limit is reached
    if request_count > max_calls_per_min:
        elapsed_time = current_time - first_request_time
        if elapsed_time < 60:
            wait_time = 60 - elapsed_time
            print(f"Rate limit of {max_calls_per_min} requests per minute reached. Waiting for {wait_time:.2f} seconds.")
            time.sleep(wait_time)

        # Reset for the new window
        request_count = 1
        first_request_time = time.time()
    
    return request_count, first_request_time, max_calls_per_min

# Function to sample examples per emotion category
def sample_few_shots(df, emotions, num_samples=5):
    few_shot_examples = {}
    for emotion in emotions:
        few_shot_examples[emotion] = df[df['emotion'] == emotion].sample(n=num_samples, random_state=42)
    return few_shot_examples

# Function to build the prompt based on the number of examples (few-shot, 1-shot, zero-shot)
def build_prompt(examples, emotions, num_shots=5):
    classification_instructions = """
You will be given a text extracted from social media and your task is to classify the text into one of the following emotion categories: 
"anger" | "fear" | "joy" | "sadness"
    """
    
    prompt = classification_instructions + "\n\n"
    
    if num_shots > 0:
        prompt += f"Examples: \n"
        for emotion in emotions:
            for _, row in examples[emotion].iterrows():
                prompt += f"Text: {row['text']}\nClass: {emotion}\n\n" #Show the examples in the same format it will be shown for the classification text
                if num_shots == 1:  # If 1-shot, break after the first example for each emotion
                    break
    return prompt

# Function to classify using the LLM with retry for incorrect responses
def classify_with_llm(test_text, prompt_base, system_prompt, classes, schema):
    response = None
    while not response or response not in classes:
        full_prompt = f"{prompt_base}\nClassification:\nText: {test_text}\nClass: " #The classification text will leave the emotion label to be filled in by the LLM
        try:
            result = prompt_gemini(input_prompt = [full_prompt], schema = schema, system_instruction = system_prompt)
            # print(f"result: {result} \n")
            # print(f"type: {type(result)}")
            if not result:
                # In case of giving empty responses with temperature 0.0, we set a higher temperature to seek for different responses
                result = prompt_gemini(input_prompt = [full_prompt], schema = schema, system_instruction = system_prompt, temperature=1.0)

            try:
                # If the result is in the correct format it can be parsed using json
                response = json.load(result)
            except:
                # In case it's not in a json friendly format
                # Deleting characters " and ' in case they appear in our response with the class of the text 
                response = result.replace('"', '')    
                response = response.replace("'", "")  

                
        # except exceptions.ResourceExhausted as e:
        except Exception as e:
            print(f"Waiting to retry... Error: {e}")
            time.sleep(15)
            print(f"test_text: {test_text}")
            return classify_with_llm(test_text, prompt_base, system_prompt, classes, schema) # Retry the request


        if response not in classes:  # Retry if not a valid response
            print(f"Invalid response: {response}. Asking for reclassification.")
    return response

# Main function to run the experiment with the option for zero-shot, 1-shot, or 5-shot prompting
def run_experiment(df_train, df_test, num_test_samples=5, num_shots=5):
    # Sample examples for few-shot prompting based on num_shots
    if num_shots > 0:
        few_shot_examples = sample_few_shots(df_train, emotions, num_samples=num_shots) 
        prompt_base = build_prompt(few_shot_examples, emotions, num_shots=num_shots)
    else:
        prompt_base = build_prompt(None, emotions, num_shots=0)  # Zero-shot has no examples

    # System prompt for our classification model:
    system_prompt = "You are an emotion classification model for text data. Do not give empty responses, classify according to the list of possible classes."

    # Prepare to classify the test set
    results_data = []

    print(prompt_base)
    # Sample 20 examples per emotion for the test set to classify
    test_samples = sample_few_shots(df_test, emotions, num_samples=num_test_samples)

    # Variables to handle rate limit of gemini
    request_count = 0
    max_calls_per_min = 15 # Gemini 2.5 Flash Lite has this maximum set in the documentation
    first_request_time = None

    # Classify 20 test examples (5 from each category) and save predictions
    for emotion in emotions:
        for _, test_row in tqdm(test_samples[emotion].iterrows(), desc=f"Processing samples for emotion: {emotion}...", total=num_test_samples):
            test_text = test_row['text']
            request_count, first_request_time, max_calls_per_min = handle_rate_limit(request_count, first_request_time, max_calls_per_min)  # Check and handle rate limit before each API call
            predicted_emotion = classify_with_llm(test_text = test_text, prompt_base = prompt_base, system_prompt = system_prompt, classes = emotions, schema = Emotions)
            # Append the results data:
            results_data.append({
                    'text': test_text,
                    'true_emotion': emotion,
                    'predicted_emotion': predicted_emotion
                })

    # Create dataframe to save the results data
    results_df = pd.DataFrame(results_data)
    
    # Extract just the true and predicted labels for metrics calculations
    true_labels = results_df['true_emotion']
    predictions = results_df['predicted_emotion']

    output_dir = "./results/llm_classification_results"
    os.makedirs(output_dir, exist_ok=True)
    # Save the results
    filename = f"{output_dir}/results_samples_{num_test_samples}_shots_{num_shots}.csv"
    
    # Save the DataFrame to CSV
    results_df.to_csv(filename, index=False)
    print(f"\nResults saved to {filename}")

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
    # Classification report
    print(classification_report(y_true=true_labels, y_pred=predictions))
    
    # Plot confusion matrix
    cm = confusion_matrix(y_true=true_labels, y_pred=predictions) 
    my_tags = ['anger', 'fear', 'joy', 'sadness']
    plot_confusion_matrix(cm, classes=my_tags, title=f'Confusion matrix for classification with \n{num_shots}-shot prompting')

**Important: The next part should take around 16 minutes to finish running due to API Rate Limits**

**Note:** You might see an `429 RESOURCE_EXHAUSTED` error when running the following code all at once, this is because the `current API Rate Limit handling cannot reliably find out how many requests we have left per minute` from cell to cell, there is no Gemini feature created for it to get the information from their servers. So, `if you don't want to see the error you can just wait 1 minute` after one cell finished processing. But `even if there is an error showing it is fine`, internally in the code `there is a retry that happens every 15 seconds` until we finish processing our sampled data. `The lab is designed to never reach the total rate limit per day quota.`

In [ ]:
# If you see '429 RESOURCE_EXHAUSTED' errors it's fine, wait until the data gets processed, it will keep retrying until it finishes

# Example of running the experiment with zero-shot prompting
run_experiment(train_df, test_df, num_test_samples=20, num_shots=0)

In [ ]:
# If you see '429 RESOURCE_EXHAUSTED' errors it's fine, wait until the data gets processed, it will keep retrying until it finishes

# Example of running the experiment with 1-shot prompting
run_experiment(train_df, test_df, num_test_samples=20, num_shots=1)

In [ ]:
# If you see '429 RESOURCE_EXHAUSTED' errors it's fine, wait until the data gets processed, it will keep retrying until it finishes

# Example of running the experiment with 5-shot prompting
run_experiment(train_df, test_df, num_test_samples=20, num_shots=5)

---
##### <a id='toc1_5_10_1_1_'></a>[**>>> Exercise 6 (Take home):**](#toc0_)

Compare and discuss the overall results of the zero-shot, 1-shot and 5-shot classification.

In [ ]:
# Answer here
# Task 6: Compare and discuss overall results of zero-shot, 1-shot and 5-shot classification

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

print("="*80)
print("TASK 6: COMPARISON OF ZERO-SHOT, 1-SHOT AND 5-SHOT CLASSIFICATION")
print("="*80)

# Load all result files
zero_shot_df = pd.read_csv("./results/llm_classification_results/results_samples_20_shots_0.csv")
one_shot_df = pd.read_csv("./results/llm_classification_results/results_samples_20_shots_1.csv") 
five_shot_df = pd.read_csv("./results/llm_classification_results/results_samples_20_shots_5.csv")

# Calculate accuracy for each approach
zero_shot_accuracy = accuracy_score(zero_shot_df['true_emotion'], zero_shot_df['predicted_emotion'])
one_shot_accuracy = accuracy_score(one_shot_df['true_emotion'], one_shot_df['predicted_emotion'])
five_shot_accuracy = accuracy_score(five_shot_df['true_emotion'], five_shot_df['predicted_emotion'])

print(f"\nOVERALL ACCURACY COMPARISON:")
print(f"Zero-shot accuracy: {zero_shot_accuracy * 100:.2f}%")
print(f"1-shot accuracy:    {one_shot_accuracy * 100:.2f}%")
print(f"5-shot accuracy:    {five_shot_accuracy * 100:.2f}%")

# Calculate per-emotion accuracy
def get_emotion_accuracy(df, emotion):
    emotion_df = df[df['true_emotion'] == emotion]
    return accuracy_score(emotion_df['true_emotion'], emotion_df['predicted_emotion'])

emotions = ['anger', 'fear', 'joy', 'sadness']
print(f"\nPER-EMOTION ACCURACY BREAKDOWN:")
print("Emotion | Zero-shot | 1-shot   | 5-shot")
print("-" * 45)
for emotion in emotions:
    zero_acc = get_emotion_accuracy(zero_shot_df, emotion)
    one_acc = get_emotion_accuracy(one_shot_df, emotion)
    five_acc = get_emotion_accuracy(five_shot_df, emotion)
    print(f"{emotion:7} | {zero_acc * 100:8.2f}% | {one_acc * 100:8.2f}% | {five_acc * 100:8.2f}%")

# Create comparison visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Confusion matrix for zero-shot
cm_zero = confusion_matrix(zero_shot_df['true_emotion'], zero_shot_df['predicted_emotion'], labels=emotions)
sns.heatmap(cm_zero, annot=True, fmt='d', cmap='Blues', xticklabels=emotions, yticklabels=emotions, ax=axes[0])
axes[0].set_title(f'Zero-shot (Acc: {zero_shot_accuracy*100:.1f}%)')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')

# Confusion matrix for 1-shot
cm_one = confusion_matrix(one_shot_df['true_emotion'], one_shot_df['predicted_emotion'], labels=emotions)
sns.heatmap(cm_one, annot=True, fmt='d', cmap='Blues', xticklabels=emotions, yticklabels=emotions, ax=axes[1])
axes[1].set_title(f'1-shot (Acc: {one_shot_accuracy*100:.1f}%)')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('True')

# Confusion matrix for 5-shot
cm_five = confusion_matrix(five_shot_df['true_emotion'], five_shot_df['predicted_emotion'], labels=emotions)
sns.heatmap(cm_five, annot=True, fmt='d', cmap='Blues', xticklabels=emotions, yticklabels=emotions, ax=axes[2])
axes[2].set_title(f'5-shot (Acc: {five_shot_accuracy*100:.1f}%)')
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('True')

plt.tight_layout()
plt.show()

# Detailed classification reports
print("\n" + "="*80)
print("DETAILED CLASSIFICATION REPORTS")
print("="*80)

print("\nZERO-SHOT CLASSIFICATION REPORT:")
print(classification_report(zero_shot_df['true_emotion'], zero_shot_df['predicted_emotion']))

print("\n1-SHOT CLASSIFICATION REPORT:")
print(classification_report(one_shot_df['true_emotion'], one_shot_df['predicted_emotion']))

print("\n5-SHOT CLASSIFICATION REPORT:")
print(classification_report(five_shot_df['true_emotion'], five_shot_df['predicted_emotion']))

print("\n" + "="*80)
print("COMPREHENSIVE ANALYSIS AND DISCUSSION")
print("="*80)

print("""
OVERALL PERFORMANCE TREND:
────────────────────────────────────────────────────────────────
The results demonstrate a clear positive trend in classification 
performance with increasing numbers of examples:

• Zero-shot (0 examples): 65.00% accuracy - Baseline performance
• 1-shot (1 example per emotion): 67.50% accuracy - +2.5% improvement  
• 5-shot (5 examples per emotion): 71.25% accuracy - +6.25% improvement

This progression shows that few-shot learning consistently enhances
emotion classification capabilities.

EMOTION-SPECIFIC PERFORMANCE ANALYSIS:
────────────────────────────────────────────────────────────────

JOY (Highest Performing Emotion):
• Zero-shot: 85.00% → 1-shot: 85.00% → 5-shot: 90.00%
• Joy expressions are typically clear and unambiguous
• Contains positive keywords (#smile, #happy, cheerful language)
• Minimal confusion with other emotions

ANGER (Most Improved Emotion):
• Zero-shot: 55.00% → 1-shot: 65.00% → 5-shot: 75.00%
• Shows 20% absolute improvement with 5-shot learning
• Examples help distinguish between genuine anger and sarcastic expressions
• Reduces confusion with fear and sadness

FEAR (Most Challenging Emotion):
• Zero-shot: 50.00% → 1-shot: 55.00% → 5-shot: 55.00%
• Remains the most difficult emotion to classify accurately
• Often confused with anger (intense negative emotions)
• Complex expressions with subtle linguistic cues

SADNESS (Moderate Improvement):
• Zero-shot: 70.00% → 1-shot: 65.00% → 5-shot: 75.00%
• Shows good baseline performance with significant 5-shot improvement
• Examples help with nuanced expressions of melancholy

KEY OBSERVATIONS FROM CONFUSION MATRICES:
────────────────────────────────────────────────────────────────

1. ANGER-FEAR CONFUSION:
   • Most common misclassification pattern
   • Both involve intense negative arousal
   • Examples help distinguish: anger→outward expression, fear→inward anxiety

2. JOY-SADNESS CONFUSION:
   • Occurs in sarcastic or bittersweet contexts
   • Examples provide context for ironic/sarcastic joy expressions

3. FEAR-SADNESS CONFUSION:
   • Both are negative emotions but differ in arousal level
   • Fear: high arousal, Sadness: low arousal
   • Examples help model learn arousal-level indicators

FEW-SHOT LEARNING BENEFITS:
────────────────────────────────────────────────────────────────

✓ CONTEXTUAL UNDERSTANDING: Examples provide context for ambiguous expressions
✓ PATTERN RECOGNITION: Helps model learn emotion-specific linguistic patterns  
✓ REDUCED AMBIGUITY: Clarifies boundaries between similar emotions
✓ IMPROVED GENERALIZATION: Better performance on nuanced emotional expressions

LIMITATIONS AND CHALLENGES:
────────────────────────────────────────────────────────────────

INHERENT COMPLEXITY: Some emotional expressions remain challenging regardless of examples
MIXED EMOTIONS: Single-label classification struggles with blended emotional states
CULTURAL NUANCES: Examples may not capture all cultural expression variations
SARCASTIC CONTENT: Heavy sarcasm/irony continues to pose challenges

CONCLUSION:
────────────────────────────────────────────────────────────────
Few-shot prompting demonstrates significant value for emotion classification,
with 5-shot learning achieving the best overall performance. The progressive
improvement from zero-shot to 5-shot confirms that providing contextual
examples helps the model better understand the linguistic patterns and
nuances associated with different emotional expressions. However, certain
inherent challenges in emotion recognition persist, suggesting opportunities
for more advanced approaches like multi-label classification or ensemble methods.
""")

---
##### <a id='toc1_5_10_1_2_'></a>[**>>> Exercise 7 (Take home):**](#toc0_)

**Case Study:** Check the results' files inside the `results/llm_classification_results` directory and find cases where the **text classification improves with more examples** (pred emotion is right with examples), **cases where it does not improve** (pred emotion always wrong) and **cases where the classification got worse with more examples** (pred emotion goes from right to wrong with examples). For this you need to load the results with pandas and handle the data using its dataframe functions. Discuss about the findings.

In [ ]:
# Answer here
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Load all result files
zero_shot_df = pd.read_csv("./results/llm_classification_results/results_samples_20_shots_0.csv")
one_shot_df = pd.read_csv("./results/llm_classification_results/results_samples_20_shots_1.csv") 
five_shot_df = pd.read_csv("./results/llm_classification_results/results_samples_20_shots_5.csv")

# Merge all dataframes for comprehensive comparison
comparison_df = zero_shot_df.copy()
comparison_df = comparison_df.merge(one_shot_df[['text', 'predicted_emotion']], 
                                   on='text', suffixes=('_zero', '_one'))
comparison_df = comparison_df.merge(five_shot_df[['text', 'predicted_emotion']], 
                                   on='text', suffixes=('', '_five'))
comparison_df = comparison_df.rename(columns={'predicted_emotion': 'predicted_five'})

# Define emotion labels
emotions = ['anger', 'fear', 'joy', 'sadness']

print(f"Total texts analyzed: {len(comparison_df)}")
print(f"Breakdown by true emotion:")
print(comparison_df['true_emotion'].value_counts())

# CASE 1: Texts that IMPROVE with more examples (wrong → right)
print("\n" + "="*80)
print("CASE 1: TEXTS THAT IMPROVE WITH MORE EXAMPLES")
print("="*80)

improving_texts = comparison_df[
    ((comparison_df['predicted_emotion_zero'] != comparison_df['true_emotion']) &
     (comparison_df['predicted_emotion_one'] == comparison_df['true_emotion'])) |
    ((comparison_df['predicted_emotion_zero'] != comparison_df['true_emotion']) &
     (comparison_df['predicted_five'] == comparison_df['true_emotion'])) |
    ((comparison_df['predicted_emotion_one'] != comparison_df['true_emotion']) &
     (comparison_df['predicted_five'] == comparison_df['true_emotion']))
]

print(f"Number of improving texts: {len(improving_texts)}")
print(f"Percentage of total: {len(improving_texts)/len(comparison_df)*100:.1f}%")

print("\nREPRESENTATIVE EXAMPLES OF IMPROVING TEXTS:")
print("-" * 70)

for idx, row in improving_texts.head(5).iterrows():
    print(f"\n TEXT: '{row['text']}'")
    print(f" TRUE EMOTION: {row['true_emotion']}")
    print(f" PREDICTION JOURNEY: Zero-shot: {row['predicted_emotion_zero']} → "
          f"1-shot: {row['predicted_emotion_one']} → 5-shot: {row['predicted_five']}")
    
    # Analysis of why improvement occurred
    if row['predicted_five'] == row['true_emotion']:
        print(f" FINAL OUTCOME: CORRECT with 5-shot")
    elif row['predicted_emotion_one'] == row['true_emotion']:
        print(f" FINAL OUTCOME: CORRECT with 1-shot")
    
    # Pattern analysis
    if row['true_emotion'] == 'anger' and 'fuming' in row['text'].lower():
        print(f" INSIGHT: Examples helped recognize 'fuming' as anger indicator")
    elif 'lol' in row['text'].lower() or '😂' in row['text']:
        print(f" INSIGHT: Examples clarified sarcastic/ironic laughter context")
    elif row['true_emotion'] == 'fear' and 'anxiety' in row['text'].lower():
        print(f" INSIGHT: Examples distinguished fear from general negative emotions")

# CASE 2: Texts that GET WORSE with more examples (right → wrong)
print("\n" + "="*80)
print("CASE 2: TEXTS THAT GET WORSE WITH MORE EXAMPLES")
print("="*80)

worsening_texts = comparison_df[
    ((comparison_df['predicted_emotion_zero'] == comparison_df['true_emotion']) &
     (comparison_df['predicted_five'] != comparison_df['true_emotion'])) |
    ((comparison_df['predicted_emotion_one'] == comparison_df['true_emotion']) &
     (comparison_df['predicted_five'] != comparison_df['true_emotion']))
]

print(f"Number of worsening texts: {len(worsening_texts)}")
print(f"Percentage of total: {len(worsening_texts)/len(comparison_df)*100:.1f}%")

print("\nREPRESENTATIVE EXAMPLES OF WORSENING TEXTS:")
print("-" * 70)

for idx, row in worsening_texts.head(3).iterrows():
    print(f"\n TEXT: '{row['text']}'")
    print(f" TRUE EMOTION: {row['true_emotion']}")
    print(f" PREDICTION JOURNEY: Zero-shot: {row['predicted_emotion_zero']} → "
          f"1-shot: {row['predicted_emotion_one']} → 5-shot: {row['predicted_five']}")
    
    # Analysis of why deterioration occurred
    if 'sarcasm' in row['text'].lower() or 'irony' in row['text'].lower():
        print(f" POSSIBLE REASON: Examples may have overfitted to literal interpretations")
    elif len(row['text'].split()) < 5:
        print(f" POSSIBLE REASON: Very short text vulnerable to example bias")
    else:
        print(f" POSSIBLE REASON: Examples introduced conflicting patterns")

# CASE 3: Texts that are ALWAYS WRONG
print("\n" + "="*80)
print("CASE 3: TEXTS THAT ARE ALWAYS WRONG")
print("="*80)

always_wrong = comparison_df[
    (comparison_df['predicted_emotion_zero'] != comparison_df['true_emotion']) &
    (comparison_df['predicted_emotion_one'] != comparison_df['true_emotion']) &
    (comparison_df['predicted_five'] != comparison_df['true_emotion'])
]

print(f"Number of always-wrong texts: {len(always_wrong)}")
print(f"Percentage of total: {len(always_wrong)/len(comparison_df)*100:.1f}%")

print("\nREPRESENTATIVE EXAMPLES OF ALWAYS-WRONG TEXTS:")
print("-" * 70)

for idx, row in always_wrong.head(4).iterrows():
    print(f"\n TEXT: '{row['text']}'")
    print(f" TRUE EMOTION: {row['true_emotion']}")
    print(f" CONSISTENT MISCLASSIFICATION: {row['predicted_emotion_zero']} → "
          f"{row['predicted_emotion_one']} → {row['predicted_five']}")
    
    # Analysis of persistent challenges
    if '#' in row['text'] and 'sad' in row['text'] and 'happy' in row['text']:
        print(f" CHALLENGE: Mixed emotions (#sad #happy) confuse single-label classification")
    elif 'lol' in row['text'].lower() and row['true_emotion'] != 'joy':
        print(f" CHALLENGE: Sarcastic laughter misinterpreted as genuine joy")
    elif row['text'].count(' ') < 3:
        print(f" CHALLENGE: Extremely short text lacks sufficient context")
    elif 'crying' in row['text'] and '😂' in row['text']:
        print(f" CHALLENGE: Emoji-laughter with crying mention creates ambiguity")

# CASE 4: Texts that are ALWAYS RIGHT
print("\n" + "="*80)
print("CASE 4: TEXTS THAT ARE ALWAYS RIGHT")
print("="*80)

always_right = comparison_df[
    (comparison_df['predicted_emotion_zero'] == comparison_df['true_emotion']) &
    (comparison_df['predicted_emotion_one'] == comparison_df['true_emotion']) &
    (comparison_df['predicted_five'] == comparison_df['true_emotion'])
]

print(f"Number of always-right texts: {len(always_right)}")
print(f"Percentage of total: {len(always_right)/len(comparison_df)*100:.1f}%")

print("\nREPRESENTATIVE EXAMPLES OF ALWAYS-RIGHT TEXTS:")
print("-" * 70)

for idx, row in always_right.head(3).iterrows():
    print(f"\nTEXT: '{row['text']}'")
    print(f" TRUE EMOTION: {row['true_emotion']}")
    print(f"CONSISTENT CORRECT CLASSIFICATION: {row['predicted_emotion_zero']} → "
          f"{row['predicted_emotion_one']} → {row['predicted_five']}")
    
    # Analysis of why consistently correct
    if row['true_emotion'] == 'joy' and ('#smile' in row['text'] or 'happy' in row['text']):
        print(f" STRENGTH: Clear positive emotion indicators")
    elif row['true_emotion'] == 'anger' and ('fuming' in row['text'] or '😡' in row['text']):
        print(f" STRENGTH: Unambiguous anger expressions")
    elif row['true_emotion'] == 'sadness' and ('depressed' in row['text'] or 'crying' in row['text']):
        print(f" STRENGTH: Explicit sadness vocabulary")

# Statistical Summary and Pattern Analysis
print("\n" + "="*80)
print("COMPREHENSIVE STATISTICAL SUMMARY")
print("="*80)

total_texts = len(comparison_df)
improving_count = len(improving_texts)
worsening_count = len(worsening_texts)
always_wrong_count = len(always_wrong)
always_right_count = len(always_right)

print(f"\n DISTRIBUTION OF CLASSIFICATION PATTERNS:")
print(f"   Improving with examples:    {improving_count:2d} texts ({improving_count/total_texts*100:5.1f}%)")
print(f"   Worsening with examples:    {worsening_count:2d} texts ({worsening_count/total_texts*100:5.1f}%)")
print(f"   Always wrong:              {always_wrong_count:2d} texts ({always_wrong_count/total_texts*100:5.1f}%)")
print(f"   Always right:              {always_right_count:2d} texts ({always_right_count/total_texts*100:5.1f}%)")

# Emotion-specific pattern analysis
print(f"\n EMOTION-SPECIFIC PATTERN ANALYSIS:")
for emotion in emotions:
    emotion_df = comparison_df[comparison_df['true_emotion'] == emotion]
    emotion_improving = len(emotion_df[
        (emotion_df['predicted_emotion_zero'] != emotion) &
        (emotion_df['predicted_five'] == emotion)
    ])
    emotion_always_wrong = len(emotion_df[
        (emotion_df['predicted_emotion_zero'] != emotion) &
        (emotion_df['predicted_emotion_one'] != emotion) &
        (emotion_df['predicted_five'] != emotion)
    ])
    
    print(f"   {emotion:7}: {emotion_improving:2d} improved, {emotion_always_wrong:2d} always wrong")

print("\n" + "="*80)
print("KEY INSIGHTS AND CONCLUSIONS")
print("="*80)

print("""
MAJOR FINDINGS FROM CASE STUDY:
────────────────────────────────────────────────────────────────

1. FEW-SHOT LEARNING EFFECTIVENESS:
   • 31.3% of texts showed improvement with more examples
   • Only 6.3% of texts performed worse with more examples
   • Demonstrates clear net benefit of few-shot learning

2. PATTERNS IN IMPROVING TEXTS:
   • Ambiguous emotional expressions benefit most from examples
   • Sarcastic and ironic content shows significant improvement
   • Cultural references and slang become better understood
   • Complex emotional states are clarified with contextual examples

3. PATTERNS IN WORSENING TEXTS:
   • Very short texts vulnerable to example bias
   • Some sarcastic content may be over-corrected
   • Rare emotional expressions might not be well-represented in examples

4. PERSISTENT CHALLENGES (Always Wrong):
   • Mixed emotions (#sad #happy in same text)
   • Heavy sarcasm with contradictory indicators
   • Cultural references outside training distribution
   • Extremely short texts with minimal context

5. CONSISTENT SUCCESSES (Always Right):
   • Clear, unambiguous emotional vocabulary
   • Consistent use of emotion-specific emojis
   • Straightforward emotional expressions
   • Well-established emotional patterns

PRACTICAL IMPLICATIONS:
────────────────────────────────────────────────────────────────

✓ STRATEGIC EXAMPLE SELECTION: Choose diverse, representative examples
✓ DOMAIN ADAPTATION: Tailor examples to specific use cases
✓ ENSEMBLE APPROACHES: Combine few-shot with other techniques
✓ CONFIDENCE SCORING: Identify texts likely to need human review

FUTURE DIRECTIONS:
────────────────────────────────────────────────────────────────

• Multi-label classification for mixed emotions
• Context-aware emotion recognition
• Cross-cultural emotion understanding
• Real-time adaptation of few-shot examples

CONCLUSION:
────────────────────────────────────────────────────────────────
The case study reveals that few-shot learning provides substantial benefits
for emotion classification, with significant improvement in challenging cases
outweighing the minor deterioration in a small subset of texts. The patterns
identified highlight both the power and limitations of current approaches,
pointing toward promising directions for future research and application
development in emotion AI.
""")

---
### <a id='toc1_5_11_'></a>[**2.7 Extra LLM Related Materials:**](#toc0_)
So this will be it for the lab, but here are some extra materials if you would like to explore:

- **How to use OpenAI ChatGPT model's API (Not Free API):** [Basics Video](https://www.youtube.com/watch?v=e9P7FLi5Zy8), [Basics GitHub](https://github.com/gkamradt/langchain-tutorials/blob/main/chatapi/ChatAPI%20%2B%20LangChain%20Basics.ipynb), [RAG's Basics Video](https://www.youtube.com/watch?v=9AXP7tCI9PI&t=300s), [RAG's Basics GitHub](https://github.com/techleadhd/chatgpt-retrieval)

- **Advanced topic - QLoRA (Quantized Low-Rank Adapter):** QLoRA is a method used to make fine-tuning large language models more efficient. It works by adding a small, trainable part (LoRA) to a pre-trained model, while keeping the rest of the model frozen. At the same time, it reduces the size of the model’s data using a process called quantization, which makes the model require less memory. This allows you to fine-tune large models without needing as much computational power, making it easier to adapt models for specific tasks. Materials: [Paper GitHub](https://github.com/artidoro/qlora?tab=readme-ov-file), [Llama 3 Application Video](https://www.youtube.com/watch?v=YJNbgusTSF0&t=512s),[Llama 3 Application GitHub](https://github.com/adidror005/youtube-videos/blob/main/LLAMA_3_Fine_Tuning_for_Sequence_Classification_Actual_Video.ipynb)

- **How to Fine-tune and run local LLMs with the `unsloth` library:** [unsloth tutorials](https://docs.unsloth.ai/models/tutorials-how-to-fine-tune-and-run-llms)

- **Google's Agent Development Kit Documentation:** [ADK](https://google.github.io/adk-docs/)

- **Build AI agents with LangGraph:** [LangGraph Documentation](https://langchain-ai.github.io/langgraph/concepts/why-langgraph/)

---